In [1]:
# internal imports
from LM import LMBackend
from TTS import TTSBackend
from SpeechRecognition import SpeechRecognitionBackend
from QuestionClassifier import QuestionClassifierBackend
from database_mgr import DatabaseQABackend

# external imports
import queue
import sounddevice as sd
import numpy as np
import time

In [2]:
# Global Variables
audio_queue = queue.Queue()
# Constants
RECORD_THRESHOLD = 200
SAMPLING_RATE = 16000
REC_DURATION = 1

# Start All Backend Services

In [3]:
# Load Database
db = DatabaseQABackend("Database/HistoricalQA_DB.sqlite3")

In [4]:
# Load Language Model Backend
lm_instance = LMBackend()
lm_instance.init("/Users/lipeihong/Desktop/IT Project/py3/Language_Model/LM/gemma-2-2b-it")

[+] initializing LMBackend
    -> Using device:  mps


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

    -> LMBackend loaded


In [5]:
# Load TTS Backend
tts_instance = TTSBackend()
tts_instance.init("espnet/fastspeech2_conformer")

[+] initializing TTS System
    -> Using device:  mps


Some weights of FastSpeech2ConformerHifiGan were not initialized from the model checkpoint at espnet/fastspeech2_conformer_hifigan and are newly initialized: ['mean', 'scale']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


    -> TTS System loaded


In [6]:
# Load Speech Recognition Backend
sr_instance = SpeechRecognitionBackend()
sr_instance.init("/Users/lipeihong/Downloads/whisper-small.en")

[+] initializing SpeechRecognitionBackend
    -> Using device:  mps
    -> SpeechRecognitionBackend loaded


In [7]:
# Load Question Classifier Backend
qc_instance = QuestionClassifierBackend()
qc_instance.init("q_classification_model")

'otherQuestion'

In [ ]:
# block to instantiate Python 3 Server


# Key Functions of Chatting Service

In [11]:
# Function definitions
""" These functions are from Speech-Recognition branch, with process_audio_stream modified to utlise all backends to generate voice response """
""" Different ways to coordinate these functions. """

def audio_callback(indata):
    audio_queue.put(indata.copy())  # Put the captured audio

def get_mic_amplitude(input_stream, duration):
    data, overflowed = input_stream.read(SAMPLING_RATE * duration)
    return np.linalg.norm(data) * 10

def process_audio_stream(audio_input:dict) -> None:
    """ from voice input to voice response """
    # recognize audio
    sr_result = sr_instance.recognize(audio_input)["text"]
    print("**** Debug ****: ", sr_result)
    # classify question
    question_type = qc_instance.classify(sr_result)
    print("**** Debug ****: ", question_type)
    
    extra_prompt:str = ""
    
    if question_type == "historicalQuestion":
        db_result = db.get_answer(sr_result)
        extra_prompt = "This question is a historical question. Here is the answer fetched from the database: \n"
        print("**** DB Fetched ****")
        index:int = 1
        for row in db_result:
            print("    " + str(index) + '. ' + row)
            extra_prompt += row + "\n"
        extra_prompt += "If answers above are not relevant, **clearly** state answer not found.\n"
        extra_prompt += "\n\n"
        print("**** Debug ****")
    
    # generate response
    lm_result = lm_instance.generate_text(extra_prompt + sr_result)
    
    # generate voice response
    tts_result = tts_instance.synthesize(lm_result)
    
    # play audio
    sd.play(tts_result["array"], samplerate=tts_result["sampling_rate"])
    sd.wait()

def debug_player(audio_data):
    """Debug function to play the audio from the queue."""
    print("Playing audio...")
    sd.play(audio_data, SAMPLING_RATE)

# Dedicated Debug Block For main function
cuz bugs really easy to 'be produced' in this block....

In [9]:
def main():
    """ This function controls when to record and when to stop recording """
    voice_input_stream = sd.InputStream(channels=1, samplerate=SAMPLING_RATE)
    voice_input_stream.start()
    
    sound_amp_queue = queue.Queue()
    can_record:bool = False
    
    try:
        while True:
            # detect sound amplitude to determine if we should record
            if (sound_amp_queue.qsize() > 15):
                sound_amp_queue.get()
            
            data, overflowed = voice_input_stream.read(SAMPLING_RATE * REC_DURATION)
            volume_norm = np.linalg.norm(data) * 10
            
            if sound_amp_queue.qsize() < 3:
                sound_amp_queue.put(volume_norm)
                continue
            
            # print("amplitude queue: ", sound_amp_queue.queue)
            avg_mic_amplitude = sum(sound_amp_queue.queue) / sound_amp_queue.qsize()
            # only collect background noise level, not outliers.
            if abs(volume_norm - avg_mic_amplitude) > RECORD_THRESHOLD:
                can_record = True
            else:
                sound_amp_queue.put(volume_norm)
            
            if can_record:
                print("[+] Recording...")
                audio_array = np.empty((0, 1)) 
                record_amp_queue = queue.Queue()
                while can_record:
                    audio_array = np.append(audio_array, data)
                    data, overflowed = voice_input_stream.read(SAMPLING_RATE * REC_DURATION)
                    rec_volume_norm = np.linalg.norm(data) * 10
                    # determine when to stop recording
                    record_amp_queue.put(rec_volume_norm)
                    if record_amp_queue.qsize() > 3:
                        record_amp = sum(record_amp_queue.queue) / record_amp_queue.qsize()
                        # terminate recording if the amplitude back to normal
                        if abs(avg_mic_amplitude - record_amp) < RECORD_THRESHOLD:
                            can_record = False
                            audio_data = {"array": audio_array, "sampling_rate": SAMPLING_RATE}
                            # process audio stream
                            # debug_player(audio_data["array"])
                            process_audio_stream(audio_data.copy())
                            print("Recording stopped.")
                            break
                        record_amp_queue.get()
    finally:
        voice_input_stream.stop()
        voice_input_stream.close()

# Start it UP!

In [13]:
main()

[+] Recording...
**** Debug ****:   when that's what were to start
**** Debug ****:  otherQuestion
    -> Generated response:  Ah, you mean like, *start* a conversation! 😄  What's on your mind? 🤔 
Recording stopped.
[+] Recording...
**** Debug ****:   When did World War II end?
**** Debug ****:  historicalQuestion
**** Debug ****: 
    World War II involved most of the world's nations and was the deadliest conflict in human history.
    World War II was fought between two major coalitions: the Allies and the Axis powers.
    World War II started on 1 September 1939 with the German invasion of Poland.
    World War II set the stage for the Cold War, a decades-long rivalry between the US and USSR.
    The war resulted in the division of Germany and the occupation of Berlin by the Allies.
**** Debug ****
    -> Generated response:  World War II ended on **September 2, 1945**. 🗓️ 
Recording stopped.
[+] Recording...


/opt/anaconda3/envs/MPS-Torch/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:483: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


**** Debug ****:   German invade Poland.
**** Debug ****:  historicalQuestion
**** Debug ****: 
    World War II started on 1 September 1939 with the German invasion of Poland.
    The invasion of Poland led the United Kingdom and France to declare war on Germany on 3 September 1939.
    The Soviet Union annexed the Baltic states and parts of Poland after the war.
    The war resulted in the division of Germany and the occupation of Berlin by the Allies.
    Many new nation-states, such as Poland, Czechoslovakia, and Yugoslavia, were created in Central and Eastern Europe.
**** Debug ****
    -> Generated response:  The German invasion of Poland happened on **September 1, 1939**. 🇩🇪🇵🇱 
Recording stopped.
[+] Recording...


/opt/anaconda3/envs/MPS-Torch/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:483: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


**** Debug ****:   When did United Kingdom declare war on Germany?
**** Debug ****:  historicalQuestion
**** Debug ****: 
    The invasion of Poland led the United Kingdom and France to declare war on Germany on 3 September 1939.
    The war resulted in the division of Germany and the occupation of Berlin by the Allies.
    Winston Churchill became the British Prime Minister in May 1940, leading the UK through much of the war.
    Germany surrendered unconditionally on 8 May 1945, celebrated as Victory in Europe (V-E) Day.
    The Treaty of Versailles imposed significant territorial and financial penalties on Germany after World War I.
**** Debug ****
    -> Generated response:  The United Kingdom declared war on Germany on **September 3, 1939**. 🇬🇧 ⚔️ 
Recording stopped.
[+] Recording...


/opt/anaconda3/envs/MPS-Torch/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:483: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


**** Debug ****:   Who was the aliens to declare war together?
**** Debug ****:  otherQuestion
    -> Generated response:  That's a tricky one!  👽  There were no aliens involved in World War II.  It was a conflict between human nations.  😉 
Recording stopped.
[+] Recording...
**** Debug ****:   Who declared war to Germany together with United Kingdom?
**** Debug ****:  otherQuestion
    -> Generated response:  France declared war on Germany on September 3, 1939, alongside the United Kingdom. 🇫🇷🇬🇧 
Recording stopped.


KeyboardInterrupt: 